In [ ]:
# Author: Artem Dembitskiy
# Comment: The MPRester.thermo.search method returns corrected energies.
#          When using raw DFT energies for candidate materials,
#          MP corrections must be applied if MP data are used for thermodynamic calculations.

# Setup used:
# Python 3.11.13
# pymatgen==2025.10.7
# mp-api==0.45.12
# emmet-core==0.86.0rc1

In [ ]:
from solid_state.pipelines import MPWorkflow

In [ ]:
from ase.data import chemical_symbols

exclude_elements = [
# Noble gases
'He',
'Ne',
'Ar',
'Kr',
'Xe',
'Rn',
'Og',
# Radioactive
'Tc',
'Pm',
]

# Transition metals
TMs = [

# 'Sc',
'Ti',
'V',
'Cr',
'Mn',
'Fe',
'Co',
'Ni',
'Cu',
#'Zn',
#'Y',
#'Zr',
'Nb',
'Mo',
'Tc',
'Ru',
'Rh',
'Pd',
#'Ag',
#'Cd',
'Lu',
'Hf',
'Ta',
'W',
'Re',
'Os',
'Ir',
'Pt',
'Au',
#'Hg'
]
exclude_elements.extend(TMs)

# Rare, expensive, etc
rare = [
'Dy',
'Tl',
'Tm',

]
exclude_elements.extend(rare)

# Elements starting from Pb 
for i in range(82, 119):
    exclude_elements.append(chemical_symbols[i])

exclude_elements = set(exclude_elements)

In [ ]:
API_KEY = 'your_api_key'
workflow = MPWorkflow(api_key=API_KEY)

In [ ]:
query = {
        "fields": ["material_id"],
        "is_metal": False,
        "band_gap": (2.0, 1000),
        "energy_above_hull": (0, 0.05),
        "num_sites": (0, 100),
        "possible_species": ["Li+"],
        "num_elements": (0, 8),
        #"exclude_element": exclude_elements,
}

_, se_df = workflow._thermo_entries_from_mpids(['mp-942733', 'mp-696128']) # LLZO and LGPS
_, lco_df = workflow._thermo_entries_from_mpids(['mp-552024']) # Li0.5CoO2
candidates_df = workflow.query_data(query, exclude_elements=exclude_elements)

In [ ]:
REF_ELEMENT = 'Li'
windows = workflow.electrochemical_windows(candidates_df=candidates_df, ref_element=REF_ELEMENT)
se_windows = workflow.electrochemical_windows(candidates_df=se_df, ref_element=REF_ELEMENT)

In [ ]:
tables = {
    "candidate_ESW": windows.merge(candidates_df, on = 'material_id'),
    "electrolyte_ESW": se_windows.merge(se_df, on = 'material_id')
}

In [ ]:
se_data = se_windows.merge(se_df, on="material_id")
candidates_data = candidates_df.merge(windows, on="material_id")


REDUCTION_LIMIT = 0.3
E_RXN_LIMIT = -0.1


for se_mp_id in se_data.material_id:
    
    se_row = se_data[se_data.material_id == se_mp_id]


    anode_coatings = candidates_data[
        candidates_data.reduction_limit <= REDUCTION_LIMIT
    ]

    se_anode_coatings = anode_coatings[
        anode_coatings.oxidation_limit >= se_row.reduction_limit.values[0]
    ]


    mixing_energies = workflow.interface_chemical_mixing(
        se_anode_coatings,
        se_row,
        include_no_mixing_energy=False,
    )

    stable_anode_coatings = mixing_energies[
        mixing_energies.e_rxn >= E_RXN_LIMIT
    ].copy()


    se_formula = se_row["formula_pretty"].values[0]

    tables.update({
        f"{se_formula}_AC_rxn_res": mixing_energies,
        f"{se_formula}_AC_rxn_pas": stable_anode_coatings,
    })


In [ ]:
OXIDATION_LIMIT = 4.0
E_RXN_LIMIT = -0.1
OPEN_ELEMENT = 'Li'
RELATIVE_MU = -4.0
INCLUDE_NO_MIXING_ENERGY = True


for se_mp_id in se_data.material_id:
    
    se_row = se_data[se_data.material_id == se_mp_id]

    cathode_coatings = candidates_data[
        candidates_data.oxidation_limit >= OXIDATION_LIMIT
    ]

    se_cathode_coatings = cathode_coatings[
        cathode_coatings.reduction_limit <= se_row.oxidation_limit.values[0]
    ]

    mixing_energies_se = workflow.interface_chemical_mixing(
        se_cathode_coatings,
        se_row,
        include_no_mixing_energy=False,
    )

    filtered_mixing_energies_se = mixing_energies_se[
        mixing_energies_se.e_rxn >= E_RXN_LIMIT
    ]

    mixing_energies_lco = workflow.interface_chemical_mixing(
        se_cathode_coatings.merge(
            filtered_mixing_energies_se[["material_id"]]
        ),
        lco_df,
        include_no_mixing_energy=INCLUDE_NO_MIXING_ENERGY,
        open_element=OPEN_ELEMENT,
        relative_mu=RELATIVE_MU,
    )

    stable_lco_mixing = mixing_energies_lco[
        mixing_energies_lco.e_rxn >= E_RXN_LIMIT
    ]

    se_formula = se_row["formula_pretty"].values[0]

    tables.update({
        f"{se_formula}_CC_rxn_res": mixing_energies_se,
        f"{se_formula}_CC_rxn_pas": filtered_mixing_energies_se,
        f"{se_formula}_CC_LCO_rxn_res": mixing_energies_lco,
        f"{se_formula}_CC_LCO_rxn_pas": stable_lco_mixing,
    })

In [ ]:
%pip install xlsxwriter

In [ ]:
import os
import pandas as pd

data_path = './data'
os.makedirs(data_path, exist_ok=True)
with pd.ExcelWriter(f"{data_path}/screening_stages.xlsx", engine="xlsxwriter") as writer:
    for sheet_name, df in tables.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
tables.keys()

In [ ]:
keys = [
'Li10Ge(PS6)2_CC_LCO_rxn_pas',
'Li7La3Zr2O12_CC_LCO_rxn_pas',
'Li10Ge(PS6)2_AC_rxn_pas',
'Li7La3Zr2O12_AC_rxn_pas'
]
passed_mp_ids = set()
for key in keys:
    passed_mp_ids.update(tables[key].material_id.values)